# A neural network "from scratch" using automatic differentiation

Inspired by
* https://sidsite.com/posts/autodiff/
* https://github.com/karpathy/micrograd and https://www.youtube.com/watch?v=VMj-3S1tku0

In this notebook we are going to implement a neural network, a Multilayer Perceptron (MLP). To train it we will build our own Automatic Differentiation System.

## Backpropagation/Reverse Mode Automatic differentiation

Later we want to calculate the gradient w.r.t. the parameters (weights and biases of all neurons). To do so, we are going to use the [chain rule](https://en.wikipedia.org/wiki/Chain_rule) - in the single variable case for a function $f(x) = f(g(x))$

$$\frac{\partial f}{\partial x} = \frac{\partial f}{\partial g}\frac{\partial g}{\partial x}$$

In the [multivariable case](https://en.wikipedia.org/wiki/Chain_rule#Multivariable_case) this becomes

$$J_{f, x} = J_{f, g}J_{g, x}$$

where $J$ is the jacobian matrix and we do a matrix multiplication in the equation above. Written in components

$$\frac{\partial f_i}{\partial x_j} = \sum_k \frac{\partial f_i}{\partial g_k}\frac{\partial g_k}{\partial x_j}$$

So we need to **sum** over all contributions of sub-terms that depend on the variable we want to calculate the partial derivative for.

<div class="alert alert-block alert-success">
    <b>Exercise:</b> Manually calculate the gradient of
  
$f(a, b) = (a + b) \cdot (a \cdot b)$

using the multivariable chain rule via $f(a, b) = c \cdot d$ with $c = a + b$ and $d = a \cdot b$
</div>

The computation graph of this looks like this:

In [ ]:
import graphviz
g = graphviz.Digraph(node_attr={"shape": "record"})
g.attr(rankdir="LR")
g.node("a", "a")
g.node("b", "b")
g.node("c", "+|c")
g.node("d", "*|d")
g.node("f", "*|f")
g.edge("a", "c")
g.edge("b", "c")
g.edge("a", "d")
g.edge("b", "d")
g.edge("c", "f")
g.edge("d", "f")
g

There are [two types of automatic differentiation](https://en.wikipedia.org/wiki/Automatic_differentiation#Two_types_of_automatic_differentiation)

* What we are doing here is **reverse mode** automatic differentiation or **backpropagation** - we accumulate the gradient by running through the computation graph in reverse order (applying the chain rule outside to inside). This is most efficient when calculating the gradient of a scalar function w.r.t. many parameters - as in NN traning.

* One can also run in **forward mode** where the gradients are accumulated while calculating the forward computation graph (applying the chain rule inside to outside). This is most useful for calculating the derivative of many outputs w.r.t. few inputs.

### `Variable` class

So, what we need is the **local gradient** of all nodes in a computation graph. To implement this we define a `Variable` class that stores both *value* and the *local gradient*:

In [ ]:
class Variable:
    def __init__(self, value, local_grad=(), name="", op=""):
        self.value = value
        self.local_grad = local_grad
        self.name = name
        self.op = op

    def __repr__(self):
        return f"Variable({self.value:.3g})"

We also gave it an optional `name` and `op` (name of the operation that produced it) attribute for visualization purposes

In [ ]:
Variable(5)

And operations on Variables that also calculate both the value ("forward pass") and the local gradient ("backward pass")

In [ ]:
def add(a, b):
    return Variable(a.value + b.value, [(a, 1), (b, 1)], op="+")

The gradient here is a *list of tuples* of `(Variable, partial_derivative)`. For the addition the partial derivative w.r.t. each input is 1.

For multiplication we get:

In [ ]:
def mul(a, b):
    return Variable(
        a.value * b.value,
        [(a, b.value), (b, a.value)],
        op="*"
    )

<div class="alert alert-block alert-success">
    <b>Question:</b> What would we need to write for division? And for the power operator?
</div>

In [ ]:
v = add(Variable(3), Variable(4))
v

In [ ]:
v.local_grad

In [ ]:
v = mul(Variable(3), Variable(5))
v

In [ ]:
v.local_grad

For convenience, we overload the `+` and `*` operators of `Variable`:

In [ ]:
Variable.__add__ = add
Variable.__mul__ = mul

such that we can do

In [ ]:
Variable(3) + Variable(4)

In [ ]:
Variable(6) * Variable(7)

To add and multiply with constant numbers we wrap our functions such that they convert numbers to `Variable` instances

In [ ]:
def wrap(f):
    def op(*args):
        new_args = []
        for arg in args:
            if not isinstance(arg, Variable):
                arg = Variable(arg)
            new_args.append(arg)
        return f(*new_args)

    return op


Variable.__add__ = wrap(Variable.__add__)
Variable.__mul__ = wrap(Variable.__mul__)

In [ ]:
Variable(3) + 4

In [ ]:
Variable(5) * 7

To also add a `Variable` from the right to a number we overload `__radd__` and `__rmul__`

In [ ]:
Variable.__radd__ = Variable.__add__
Variable.__rmul__ = Variable.__mul__

In [ ]:
7 * Variable(5)

In [ ]:
4 + Variable(3)

### Computation graph

Now let's visualize the graph of the computation you did before $f(a, b) = c \cdot d$ with $c = a + b$ and $d = a \cdot b$

In [ ]:
from itertools import count

import graphviz

In [ ]:
def format_var(var):
    out = f"{var.value:.3g}"
    if var.name:
        out = f"{var.name}={out}"
    if var.op:
        out = f"{var.op}|{out}"
    return out

In [ ]:
def draw_graph(variable):
    g = graphviz.Digraph(node_attr={"shape": "record", "height": ".1"})
    g.attr(rankdir="LR")
    counter = count(0)
    nodes = {}

    def add_node(variable):
        node_name = f"node{next(counter)}"
        g.node(node_name, format_var(variable))
        nodes[variable] = node_name

    def add_edges(variable):
        if variable not in nodes:
            add_node(variable)
        for child_variable, deriv in variable.local_grad:
            if child_variable not in nodes:
                add_node(child_variable)
            g.edge(nodes[child_variable], nodes[variable])
            add_edges(child_variable)

    add_edges(variable)
    return g

In [ ]:
a = Variable(3, name="a")
b = Variable(4, name="b")
c = a + b; c.name = "c"
d = a * b; d.name = "d"
f = c * d; f.name = "f"

In [ ]:
draw_graph(f)

To get the gradient of Variable `f` we need to run backwards through this graph, always summing the gradients of all paths leading to a node.

Therefore we need to process the graph in **reverse [topological order](https://en.wikipedia.org/wiki/Topological_sorting)** - where a topological ordering means dependencies of nodes have to come before nodes that depend on them.

In [ ]:
def topo_ordered_nodes(variable):
    nodes = []
    visited = set()

    def add_nodes(variable):
        if variable in visited:
            return
        visited.add(variable)
        for child_variable, deriv in variable.local_grad:
            add_nodes(child_variable)
        nodes.append(variable)

    add_nodes(variable)

    return nodes

In [ ]:
topo_ordered_nodes(f)

### The backpropagation algorithm

Now the gradient function - the **backpropagation**:

In [ ]:
from collections import defaultdict

In [ ]:
def get_gradient(variable):
    grad = defaultdict(int)
    grad[variable] = 1
    for parent_variable in reversed(topo_ordered_nodes(variable)):
        for child_variable, deriv in parent_variable.local_grad:
            grad[child_variable] += deriv * grad[parent_variable]
    return grad

In [ ]:
get_gradient(f)

We can also approximate the derivative numerically:

In [ ]:
def f(a, b):
    c = a + b
    d = a * b
    return c * d

In [ ]:
dx = 1e-5
dy = f(3 + dx, 4) - f(3, 4)
dy / dx

In [ ]:
dx = 1e-5
dy = f(3, 4 + dx) - f(3, 4)
dy / dx

<div class="alert alert-block alert-success">
    <b>Question:</b> Why then even use backpropagation and not just numerical gradients?

<details>
<summary>Answer</summary>
    <ol>
        <li>
            It can be more efficient since we only need to (reverse) traverse the computation graph <b>once</b> to get the gradient w.r.t. <b>all</b> variables. For numerical derivatives we need at least one function evaluation per variable.
        </li>
        <li>We get <b>exact</b> derivatives
    </ol>
</details>

</div>

### Neural network in terms of `Variable` instances

To make a neural network, we start with a `Neuron` that has its `weights` and the `bias` defined in terms of `Variable` instances

In [ ]:
import random

In [ ]:
class Neuron:
    def __init__(self, n_inputs):
        self.n_inputs = n_inputs
        self.weights = [Variable(random.normalvariate(0, 1)) for _ in range(n_inputs)]
        self.bias = Variable(0)

    def __call__(self, inputs):
        return sum(x * weight for x, weight in zip(self.weights, inputs)) + self.bias

We initialized the weights to random numbers around 0 which is a common practice. There are some arguments for using specific rules for how exactly to scale these, but for the purpose of this demonstration it's enough to just use a standard normal distribution.

In [ ]:
x = [random.random() for _ in range(3)]

In [ ]:
neuron = Neuron(3)

In [ ]:
neuron(x)

In [ ]:
draw_graph(neuron(x))

The sum looks a bit ugly (we have all the additions as individual operations in there)

so let's introduce sum as an operation:

In [ ]:
def var_sum(*variables):
    return Variable(sum(v.value for v in variables), [(v, 1) for v in variables], op="sum")

In [ ]:
def new_call(self, inputs):
    return (
        var_sum(*[x * weight for x, weight in zip(self.weights, inputs)]) + self.bias
    )

Neuron.__call__ = new_call

In [ ]:
neuron = Neuron(3)

In [ ]:
draw_graph(neuron(x))

Next, we need the [activation functions](NN_Activation.ipynb) - we will use **ReLU** for the hidden layers:

In [ ]:
def relu(variable):
    return Variable(
        max(variable.value, 0),
        [(variable, int(variable.value > 0))],
    )

And the **sigmoid** activation function for the final layer since we are going to solve a binary classification problem.

A useful way to write the [derivative of the sigmoid function](https://en.wikipedia.org/wiki/Logistic_function#Derivative) is in terms of the function value itself:

$y = \frac{1}{1 + e^{-x}}$

$\frac{\partial y}{\partial x} = y \cdot (1 - y)$

In [ ]:
from math import exp, log

In [ ]:
def sigmoid(var):
    x = var.value
    y = 1 / (1 + exp(-x))
    return Variable(y, [(var, y * (1 - y))])

With this we define our neural network. A layer is just a list of neurons that receive the same inputs:

In [ ]:
class Layer:
    def __init__(self, n_inputs, n_outputs):
        self.neurons = [Neuron(n_inputs) for _ in range(n_outputs)]
        
    def __call__(self, inputs):
        return [neuron(inputs) for neuron in self.neurons]

And a Multilayer Perceptron - **MLP** - a stack of Layers with activation fuctions after each Neuron:

In [ ]:
class MLP:
    def __init__(self, n_inputs, neurons_per_layer):
        self.neurons_per_layer = neurons_per_layer
        self.layers = []
        for n_outputs in neurons_per_layer:
            self.layers.append(Layer(n_inputs, n_outputs))
            n_inputs = n_outputs

    def __call__(self, inputs):
        x = inputs
        for layer in self.layers:
            x = layer(x)
            if layer is not self.layers[-1]:
                x = [relu(xi) for xi in x]
        return [sigmoid(xi) for xi in x]

In [ ]:
mlp = MLP(5, [2, 1])

In [ ]:
draw_graph(mlp(x)[0])

### Training

We will use the binary cross entropy loss to train the model for a binary classification problem:

In [ ]:
@wrap
def binary_crossentropy(var_y_true, var_y_pred):
    y_true = var_y_true.value
    y_pred = var_y_pred.value
    eps = 1e-50 # to avoid math domain errors from log(0)
    return Variable(
        -(y_true * log(y_pred + eps) + (1 - y_true) * log(1 - y_pred + eps)),
        [(var_y_pred, -y_true / (y_pred + eps) + (1 - y_true) / (1 - y_pred + eps))],
    )

Note that we skipped the gradient w.r.t. `y_true` because we don't need that for backpropagation

In [ ]:
y_pred = mlp(x)[0]

In [ ]:
y_true = 1

In [ ]:
loss = binary_crossentropy(y_true, y_pred)

In [ ]:
grad = get_gradient(loss)

In [ ]:
grad

Again, let's test a few derivatives numerically:

In [ ]:
i = 0
j = 0
k = 0
grad[mlp.layers[i].neurons[j].weights[k]]

In [ ]:
def f(weight):
    old_weight = mlp.layers[i].neurons[j].weights[k]
    mlp.layers[i].neurons[j].weights[k].value = weight
    y_pred = mlp(x)[0]
    mlp.layers[i].neurons[j].weights[k] = old_weight
    return binary_crossentropy(y_true, y_pred).value

In [ ]:
weight = mlp.layers[i].neurons[j].weights[k].value
dx = 1e-5
dy = f(weight + dx) - f(weight)
dy / dx

Now the gradient update function:

In [ ]:
def update(mlp, grad, learning_rate=1):
    for layer in mlp.layers:
        for neuron in layer.neurons:
            for weight in neuron.weights + [neuron.bias]:
                weight.value -= grad[weight] * learning_rate

And the training loop - we will use *stochastic gradient decsent* (SGD) with batches:

In [ ]:
def fit(mlp, X, y, epochs=10, batch_size=32, learning_rate=1):
    for epoch in range(epochs):
        for start in range(0, len(X), batch_size):
            x_batch = X[start : start + batch_size]
            y_batch = y[start : start + batch_size]
            outputs = [mlp(x)[0] for x in x_batch]
            losses = [
                binary_crossentropy(y_true, y_pred)
                for y_true, y_pred in zip(y_batch, outputs)
            ]
            loss = var_sum(*losses) * (1 / len(losses)) # mean
            grad = get_gradient(loss)
            update(mlp, grad, learning_rate=1)
        print(f"{epoch=}, {loss.value=}")
    return outputs

Let's try to train on the *moons* dataset again:

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn.datasets import make_moons

In [ ]:
X, y = make_moons(1000, noise=0.2)

In [ ]:
plt.plot(*X[y == 0].T, ".", label="0")
plt.plot(*X[y == 1].T, ".", label="1")
plt.legend(title="label")

In [ ]:
mlp = MLP(2, [16, 1])

In [ ]:
outputs = fit(mlp, X, y, epochs=10)

In [ ]:
xx, yy = np.meshgrid(
    np.linspace(min(X[:, 0]), max(X[:, 0]), 100),
    np.linspace(min(X[:, 1]), max(X[:, 1]), 100)
)

In [ ]:
xy = np.stack([xx, yy], axis=-1).reshape(-1, 2)

In [ ]:
z = np.array([mlp(xi)[0].value for xi in xy])

In [ ]:
plt.contourf(xx, yy, z.reshape(xx.shape), cmap="RdBu", alpha=0.8)
plt.scatter(*X.T, marker=".", c=y, cmap="RdBu")

## Extra: Using arrays

It's not too difficult to extend this system to use `numpy` arrays as input and output for each node in the computation graph.

Recall the multivariable chain rule:

$$\frac{\partial f_i}{\partial x_j} = \sum_k \frac{\partial f_i}{\partial g_k}\frac{\partial g_k}{\partial x_j}$$

That means if we have a vector at each node, we would in theory need to store the local Jacobians at each stage. However looking at the formula for the gradient of a single scalar output (fixed $i$) $\frac{\partial f}{\partial \vec x}$ we see that we actually only need the product of the gradient vector $\frac{\partial f}{\partial \vec g}$ with the jacobian $\frac{\partial \vec g}{\partial \vec x}$ - the **vector jacobian product**

In [ ]:
class Tensor:
    def __init__(self, array, backward=None, op=None, name=None):
        self.array = array
        self.backward = None
        self.op = op
        self.name = name
        
    def __repr__(self):
        prefix = "Tensor("
        lines = repr(self.array).splitlines()
        for i in range(1, len(lines)):
            lines[i] = " " * len(prefix) + lines[i]
        array_repr = "\n".join(lines)
        return f"{prefix}{array_repr})"
    
    def __add__(self, other):
        out = Tensor(self.array + other.array)
        
        def backward(grads):
            return [(self, grads[self]), (other, grads[other])]

In [ ]:
import numpy as np

In [ ]:
Tensor(np.random.rand(5, 3))